# Create Metadata Views
### Mostly for convenience

In [ ]:
%addconnect name=Transcend, user=sh186014, host=tdprdcop3.td.teradata.com, props="LOGMECH=LDAP"

In [ ]:
%connect name=Transcend

In [ ]:
database adlste_coa;

In [ ]:
SELECT * FROM  TABLE (SYSLIB.MonitorSystemPhysicalConfig()) AS SystemConfig;

<br/><br/><br/>
### meta_objects

In [ ]:
DROP TABLE ADLSTE_COA.coat_objects 

In [ ]:
CREATE MULTISET TABLE ADLSTE_COA.coat_objects 
(DatabaseName  VARCHAR(128) CHARACTER SET LATIN NOT CASESPECIFIC 
,TableName     VARCHAR(128) CHARACTER SET LATIN NOT CASESPECIFIC 
,TableKind     CHAR(1)      CHARACTER SET LATIN NOT CASESPECIFIC 
,ObjType       VARCHAR(30)  CHARACTER SET LATIN NOT CASESPECIFIC 
,ObjFunction   VARCHAR(30)  CHARACTER SET LATIN NOT CASESPECIFIC 
,ProcessName   VARCHAR(30)  CHARACTER SET LATIN NOT CASESPECIFIC
) NO PRIMARY INDEX ;

In [ ]:
replace PROCEDURE adlste_coa.sp_objects ()
   DYNAMIC RESULT SETS 1
BEGIN
    delete from adlste_coa.coat_objects;
    
    insert into adlste_coa.coat_objects
    select databasename, tablename, tablekind 
    ,STRTOK(TableName,'_',1) as ObjType
    ,STRTOK(TableName,'_',2) as ObjFunction
    ,substr(TableName, character_Length(ObjType)+character_Length(ObjFunction)+3 ) as ProcessName
    from dbc.tablesV 
    where databasename like 'adlste_coa%';

    /* return this resultset from the SP: */
    BEGIN
        DECLARE cur1 CURSOR WITH RETURN ONLY FOR
            SELECT * FROM adlste_coa.coat_objects
            order by ProcessName,ObjFunction, ObjType;
        OPEN cur1;
    END;

END;

In [16]:
Select * from adlste_coa.coa_objects
where databasename = 'adlste_coa'
and tablename like '%db_object%'

DatabaseName,TableName,TableKind,ObjType,ObjFunction,ProcessName
ADLSTE_COA,sp_dat_DB_Objects,P,sp,dat,DB_Objects
ADLSTE_COA,coat_dat_DB_Objects,T,coat,dat,DB_Objects
ADLSTE_COA,coat_dat_DB_Objects_bkup,T,coat,dat,DB_Objects_bkup
ADLSTE_COA,coa_dat_DB_Objects,V,coa,dat,DB_Objects


In [ ]:
call adlste_coa.sp_objects() 

In [ ]:
Replace View adlste_coa.coa_objects as
locking row for access
Select * from adlste_coa.coat_objects;

In [ ]:
Select * from adlste_coa.coa_objects
where tablename like '%feature%'
order by TableName,  ObjType desc;

<br/><br/><br/>
### Reconcile Column Names

In [ ]:
Select columnName, count(*) from dbc.columnsV
where databasename like 'adlste_coa%'
group by 1
order by 1